In [14]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [26]:
import os
import re
import numpy as np
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from pprint import pprint
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.models import LdaModel

In [42]:
# Ensure nltk stopwords are downloaded
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

# Set up stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
def read_txt_to_paragraphs(file_path):
    """
    Read a text file and split it into paragraphs based on empty lines.

    :param file_path: Path to the text file.
    :return: List of paragraphs, where each paragraph is a string.
    """
    with open(file_path, 'r') as file:
        content = file.read()

    # Split the content into paragraphs based on empty lines
    paragraphs = content.split('\n\n')

    # Remove any leading/trailing whitespace from each paragraph
    paragraphs = [paragraph.strip() for paragraph in paragraphs if paragraph.strip()]

    return paragraphs

# Example usage
file_path = '/content/IPCC_AR6_SYR_SPM.txt'
paragraphs_list = read_txt_to_paragraphs(file_path)

list_policy= paragraphs_list[4:]
combined_policy = '\n\n'.join(list_policy)

In [37]:
# Function to preprocess text
def preprocess_text(text):
    """
    Preprocess text by removing punctuation, stopwords, and performing tokenization.
    """
    text = re.sub('\s+', ' ', text)  # Remove newlines
    text = re.sub("\'", "", text)    # Remove single quotes
    text = gensim.utils.simple_preprocess(str(text), deacc=True)  # Tokenization and lowercasing
    text = [word for word in text if word not in stop_words]  # Remove stopwords
    return text


# Function to prepare data for LDA
def prepare_data(data):
    """
    Prepare data for LDA model.
    """
    # Preprocess text
    data_processed = list(map(preprocess_text, data))

    # Create Dictionary
    id2word = corpora.Dictionary(data_processed)

    # Create Corpus: Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in data_processed]

    return corpus, id2word

# Function to build LDA model
def build_lda_model(corpus, id2word, num_topics):
    """
    Build LDA model.
    """
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=id2word,
                                                num_topics=num_topics,
                                                random_state=100,
                                                update_every=1,
                                                chunksize=10,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)
    return lda_model

# Function to compute coherence score
def compute_coherence_score(lda_model, texts, id2word):
    """
    Compute coherence score of LDA model.
    """
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

def main():
    data = cleaned_policy

    # Prepare data
    corpus, id2word = prepare_data(data)

    # Build LDA model
    lda_model = build_lda_model(corpus, id2word, num_topics=2)

    # Print topics
    pprint(lda_model.print_topics())

    # Compute coherence score
    coherence_score = compute_coherence_score(lda_model, list(map(preprocess_text, data)), id2word)
    print('\nCoherence Score: ', coherence_score)

if __name__ == '__main__':
    main()

[(0,
  '0.020*"high" + 0.017*"climate" + 0.017*"confidence" + 0.012*"adaptation" + '
  '0.011*"global" + 0.010*"risks" + 0.009*"change" + 0.008*"financial" + '
  '0.007*"impacts" + 0.007*"regions"'),
 (1,
  '0.020*"confidence" + 0.019*"emissions" + 0.018*"high" + 0.014*"mitigation" '
  '+ 0.014*"climate" + 0.011*"development" + 0.009*"adaptation" + '
  '0.009*"global" + 0.008*"co" + 0.008*"warming"')]

Coherence Score:  0.36008899318979337


In [43]:
def extract_keywords(text, num_keywords=10):
    words = word_tokenize(text.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    word_counts = Counter(filtered_words)
    return word_counts.most_common(num_keywords)

keywords = extract_keywords(combined_policy)
print(keywords)


[('high', 267), ('confidence', 256), ('emissions', 190), ('climate', 188), ('global', 177), ('adaptation', 142), ('warming', 137), ('mitigation', 106), ('change', 97), ('figure', 96)]


In [44]:
def prepare_data_for_lda(text):
    words = [word for word in word_tokenize(text.lower()) if word.isalnum() and word not in stop_words]
    dictionary = corpora.Dictionary([words])
    corpus = [dictionary.doc2bow(words)]
    return corpus, dictionary

corpus, dictionary = prepare_data_for_lda(combined_policy)
lda_model = LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)
topics = lda_model.print_topics(num_words=4)
for topic in topics:
    print(topic)


(0, '0.020*"high" + 0.019*"confidence" + 0.014*"emissions" + 0.014*"climate"')
(1, '0.001*"high" + 0.001*"confidence" + 0.001*"climate" + 0.001*"global"')
(2, '0.001*"emissions" + 0.001*"confidence" + 0.001*"high" + 0.001*"global"')
